In [1]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd

In [3]:
df = pd.read_csv('../file/active_timeseries_prov.csv')
df['date_active'] = pd.to_datetime(df['date_active'], format='%d-%m-%Y')
df['month'] = pd.DatetimeIndex(df['date_active']).month
df['date'] = df['date_active'].dt.strftime('%Y-%m') 
# locate end of month
df['EndOfMonth'] = pd.to_datetime(df['date'], format="%Y-%m") + MonthEnd(1)
df.head()

,province,date_active,cumulative_cases,cumulative_recovered,cumulative_deaths,active_cases,active_cases_change,month,date,EndOfMonth
0,Alberta,2020-01-25,0,0,0,0,0,1,2020-01,2020-01-31
1,Alberta,2020-01-26,0,0,0,0,0,1,2020-01,2020-01-31
2,Alberta,2020-01-27,0,0,0,0,0,1,2020-01,2020-01-31
3,Alberta,2020-01-28,0,0,0,0,0,1,2020-01,2020-01-31
4,Alberta,2020-01-29,0,0,0,0,0,1,2020-01,2020-01-31


In [4]:
df_result = df.loc[(df['province'].isin(['Alberta', 'BC', 'Manitoba', 'New Brunswick', 'NL'])) &
      (df['date_active'] == df['EndOfMonth'])]
df_result1 = df.loc[(df['date_active']==df['date_active'].max()) & (df['province'].isin(['Alberta', 'BC', 'Manitoba', 'New Brunswick', 'NL']))]
df_result2 = pd.concat([df_result, df_result1])



In [5]:
df_result2.head()

,province,date_active,cumulative_cases,cumulative_recovered,cumulative_deaths,active_cases,active_cases_change,month,date,EndOfMonth
6,Alberta,2020-01-31,0,0,0,0,0,1,2020-01,2020-01-31
35,Alberta,2020-02-29,0,0,0,0,0,2,2020-02,2020-02-29
66,Alberta,2020-03-31,754,120,9,625,37,3,2020-03,2020-03-31
96,Alberta,2020-04-30,5355,2161,89,3105,-20,4,2020-04,2020-04-30
127,Alberta,2020-05-31,7010,6283,143,584,-20,5,2020-05,2020-05-31


In [6]:
df_result2.to_csv('../static/file/line_cases.csv', index=False)

In [14]:
df_Jan = df.loc[(df['month']==6)]
df_proc = df_Jan.loc[df_Jan['date_active'] == df_Jan['date_active'].max()]
df_proc.sort_values(by=['active_cases'], ascending=False, inplace=True)
df_proc.head()

/Users/denisechen/Desktop/Denise/Python/Blog_Post/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,province,date_active,cumulative_cases,cumulative_recovered,cumulative_deaths,active_cases,active_cases_change,month,date,EndOfMonth
1627,Quebec,2020-06-20,54674,23201,5408,26065,-138,6,2020-06,2020-06-30
1331,Ontario,2020-06-20,34922,28468,2640,3814,-63,6,2020-06,2020-06-30
147,Alberta,2020-06-20,7673,6996,152,525,13,6,2020-06,2020-06-30
295,BC,2020-06-20,2790,2444,168,178,0,6,2020-06,2020-06-30
1923,Saskatchewan,2020-06-20,726,639,13,74,8,6,2020-06,2020-06-30


In [15]:
df_bar = df_proc.head(5)
df_bar

,province,date_active,cumulative_cases,cumulative_recovered,cumulative_deaths,active_cases,active_cases_change,month,date,EndOfMonth
1627,Quebec,2020-06-20,54674,23201,5408,26065,-138,6,2020-06,2020-06-30
1331,Ontario,2020-06-20,34922,28468,2640,3814,-63,6,2020-06,2020-06-30
147,Alberta,2020-06-20,7673,6996,152,525,13,6,2020-06,2020-06-30
295,BC,2020-06-20,2790,2444,168,178,0,6,2020-06,2020-06-30
1923,Saskatchewan,2020-06-20,726,639,13,74,8,6,2020-06,2020-06-30


In [16]:
df_bar.to_csv('../static/file/Bar_cases.csv', index=False)

## Take the final day on the COVID data and calculate the mean of active_cases_change

In [17]:
df_result = df.loc[(df['date_active'] == df['EndOfMonth']) | (df['date_active']==df['date_active'].max())]

In [18]:
df_1 = df.groupby(['province', 'date'])['active_cases_change'].mean().reset_index()

In [19]:
df_merge = pd.merge(df_1, df_result, on=['province', 'date'])
df_merge.drop(['active_cases_change_y'], axis=1, inplace=True)
df_merge.rename(columns={"active_cases_change_x": "active_cases_mean"}, inplace=True)
df_merge.head()

,province,date,active_cases_mean,date_active,cumulative_cases,cumulative_recovered,cumulative_deaths,active_cases,month,EndOfMonth
0,Alberta,2020-01,0.000000,2020-01-31,0,0,0,0,1,2020-01-31
1,Alberta,2020-02,0.000000,2020-02-29,0,0,0,0,2,2020-02-29
2,Alberta,2020-03,20.161290,2020-03-31,754,120,9,625,3,2020-03-31
3,Alberta,2020-04,82.666667,2020-04-30,5355,2161,89,3105,4,2020-04-30
4,Alberta,2020-05,-81.322581,2020-05-31,7010,6283,143,584,5,2020-05-31


In [22]:
df_merge.to_csv('../static/file/Line_Bar.csv', index=False)

In [23]:
df_merge.head(6)

,province,date,active_cases_mean,date_active,cumulative_cases,cumulative_recovered,cumulative_deaths,active_cases,month,EndOfMonth
0,Alberta,2020-01,0.000000,2020-01-31,0,0,0,0,1,2020-01-31
1,Alberta,2020-02,0.000000,2020-02-29,0,0,0,0,2,2020-02-29
2,Alberta,2020-03,20.161290,2020-03-31,754,120,9,625,3,2020-03-31
3,Alberta,2020-04,82.666667,2020-04-30,5355,2161,89,3105,4,2020-04-30
4,Alberta,2020-05,-81.322581,2020-05-31,7010,6283,143,584,5,2020-05-31
5,Alberta,2020-06,-2.950000,2020-06-20,7673,6996,152,525,6,2020-06-30
